<a href="https://colab.research.google.com/github/aecins/tutorials/blob/main/bilinear_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2

# Load a grayscale image.
!wget "https://dome.mit.edu/bitstream/handle/1721.3/195767/cameraman.tif"
image_fullsize = cv2.imread("cameraman.tif", cv2.IMREAD_GRAYSCALE)

--2024-02-22 10:36:55--  https://dome.mit.edu/bitstream/handle/1721.3/195767/cameraman.tif
Resolving dome.mit.edu (dome.mit.edu)... 3.224.85.112, 44.220.109.227
Connecting to dome.mit.edu (dome.mit.edu)|3.224.85.112|:443... connected.
HTTP request sent, awaiting response... 200 200
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 65240 (64K) [image/tiff]
Saving to: ‘cameraman.tif.2’

cameraman.tif.2     100%[===================>]  63.71K   370KB/s    in 0.2s    

2024-02-22 10:36:56 (370 KB/s) - ‘cameraman.tif.2’ saved [65240/65240]



In [ ]:
import plotly.express as px

# Crop image
# image = image_fullsize[55:90, 135:170]   # Crop image
image = image_fullsize[62:72, 153:163]   # Crop image
# image = image_fullsize[62:64, 153:155]   # Crop image

# Show image.
fig = px.imshow(image, binary_string=True)
fig.show()

In [ ]:
import numpy as np

# Sample points between image pixels.
sample_coordinates = np.mgrid[1:image.shape[0], 1:image.shape[1]].reshape(2,-1).T
offsets = np.random.uniform(-0.5,0.5,sample_coordinates.shape)
# offsets = np.ones(sample_coordinates.shape) * 0.49999
# offsets = np.zeros(sample_coordinates.shape)
sample_coordinates = sample_coordinates + offsets

# Display sampled points.
# NOTE: there are two common image coordinate conventions:
# - integer center      : first pixel center is at (0.0, 0.0)
# - half-integer center : first pixel center is at (0.5, 0.5)
# numpy uses half-integer center convention. Thus all our processing using numpy
# is done using half-integer convention.
# plotly uses integer center convention. Thus when displaying results we must
# convert to integer convention.

xs = sample_coordinates[:, 0] - 0.5
ys = sample_coordinates[:, 1] - 0.5

fig = px.imshow(image, binary_string=True)

for x in range(image.shape[0]):
  fig.add_scatter(x=[x, x], y=[image.shape[1]-1, 0], mode='lines', line=dict(color='red'), showlegend=False)
for y in range(image.shape[1]):
  fig.add_scatter(x=[0, image.shape[0]-1], y=[y, y], mode='lines', line=dict(color='red'), showlegend=False)

fig.add_scatter(x=xs, y=ys, mode='markers', marker=dict(color='green'))

fig.show()

In [ ]:
# Use bilinear interpolation to evaluate intensity values at each sample point.
def bilinear_sample(array, sample_points):
  values = []

  for sample_point in sample_points:
    x = int(np.floor(sample_point[0]-0.5))
    y = int(np.floor(sample_point[1]-0.5))
    assert x < array.shape[0]
    assert y < array.shape[1]

    offset_x = sample_point[0] - (x + 0.5)
    offset_y = sample_point[1] - (y + 0.5)
    assert offset_x < 1
    assert offset_x < 1
    assert offset_x > 0
    assert offset_x > 0

    f_00 = array[y, x]      # top left
    f_01 = array[y, x+1]    # top right
    f_10 = array[y+1, x]    # bottom left
    f_11 = array[y+1, x+1]  # bottom right

    value = f_00 * (1-offset_x) * (1-offset_y) + f_11 * offset_x * offset_y + f_01 * offset_x * (1-offset_y) + f_10 * (1-offset_x) * offset_y
    values.append(value)

  return values

sampled_values = bilinear_sample(image, sample_coordinates)


In [ ]:
# Plot image with overlaid sampled points.
# NOTE: there is a slight mismatch between colors of the image and colors of the
# samples. I suspect that this is because the differences in how intensity is
# converted to grayscale in px.imshow() vs add_scatter().
fig = px.imshow(image, binary_string=True)

for x in range(image.shape[0]):
  fig.add_scatter(x=[x, x], y=[image.shape[1]-1, 0], mode='lines', line=dict(color='red'), showlegend=False)
for y in range(image.shape[1]):
  fig.add_scatter(x=[0, image.shape[0]-1], y=[y, y], mode='lines', line=dict(color='red'), showlegend=False)

sampled_value_colors = ['rgb({},{},{})'.format(round(x), round(x), round(x)) for x in sampled_values]

fig.add_scatter(
    x=xs,
    y=ys,
    mode='markers',
    marker=dict(
        size=20,
        color=sampled_value_colors,
    )
)
fig.show()